In [1]:
# import brightway2 as bw
import bw2calc as bc
import bw2data as bd
import numpy as np
import json, os
from scipy.stats import norm
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Local files
from import_databases import *

In [ ]:
project = "GSA for protocol"
bd.projects.set_current(project)

In [2]:
[m for m in bd.methods if 'IPCC 2013' in str(m)]

[('IPCC 2013', 'climate change', 'GTP 100a'),
 ('IPCC 2013', 'climate change', 'GTP 20a'),
 ('IPCC 2013', 'climate change', 'GWP 100a'),
 ('IPCC 2013', 'climate change', 'GWP 20a'),
 ('IPCC 2013 no LT', 'climate change', 'GTP 100a'),
 ('IPCC 2013 no LT', 'climate change', 'GTP 20a'),
 ('IPCC 2013 no LT', 'climate change', 'GWP 100a'),
 ('IPCC 2013 no LT', 'climate change', 'GWP 20a')]

In [ ]:
%%time
co = bd.Database('CH consumption 1.0')
demand_act = co.search('food sector')[0]
# demand_act = co.search('beef')[0]
demand = {demand_act: 1}
method = ('IPCC 2013', 'climate change', 'GWP 100a')
print(demand_act)

lca = bc.LCA(demand, method)
lca.lci()
lca.lcia()
lca.score

# Contribution analysis

In [ ]:
num_substances = 20
characterized_substances = lca.characterized_inventory.toarray().sum(axis=1)
inds = np.argsort(abs(characterized_substances))[::-1][:num_substances]
characterized_substances[inds]

In [ ]:
sum(characterized_substances[inds])

In [ ]:
codes = []
for ind in inds:
    act = bd.get_activity(lca.reverse_dict()[2][ind])
    codes.append((act['database'], act['code']))
    print(act)

# Uncertainties from IPCC 2013 report and it's supplementary materials (SM)

In [ ]:
# ghg_std_multiplier_dict = {}

# # 1. Time horizon, same for all cases, already specified in the LCIA method naming
# H = int( re.findall(r'\d+', method[-1])[0]) 

# # 2. Conversion from ppbv (part per billion), page 8SM-15
# Ma = 28.97      # [kg/kmol], molecular weight of air 
# Tm = 5.1352e18  # [kg] total mass of atmosphere
# ppbv_to_kg = lambda M: (Ma/M*1e9/Tm) # conversion, where M is molecular weight of the considered substance

# # 3. Compute function f uncertainty given derivatives d = partial_df/partial_dx*delta_x, and normalize
# compute_delta_f = lambda nominal_value, *d: np.sqrt(np.sum(np.power(d,2))) / nominal_value

# # 4. General expression for AGWP, equation 8.SM.9
# #   A    - radiative efficiency, [W/m2/ppb]
# #   tau  - lifetime, [yr]
# #   H    - time horizon, [yr]
# #   mult - multiplier specific to each substance, in many cases is equal to 1
# compute_agwp_general  = lambda A, tau, mult: A*tau*(1-np.exp(-H/tau)) * mult

# # 5. Print computed +-delta_x uncertainty values as percentage of the expected value \bar{x} 
# # for a 90% confidence interval and its standard deviation
# print_uncertainties = lambda x_name, x_delta, x_std: print(
#     "{:10s} -> delta = {:6.3f}%, std = {:5.3f}*mean".format(x_name, x_delta*100, x_std)
# )

# # 6. Compute standard deviation x_std given +-delta_x values for confidence interval with a certain confidence level
# conf_interval_to_std = lambda conf_level, conf_interval: conf_interval / norm.ppf(0.5 + conf_level / 2)

# def compute_co2_agwp(time_horizon):
#     # 1. Carbon dioxide, AGWP expression is not general, instead taken from equation 8.SM.23
#     # Constants are from Joos et al, 2013, https://doi.org/10.5194/acp-13-2793-2013
#     # time_horizon is given in years
#     M_co2 = 44.01
#     A_co2 = 1.37e-5 * ppbv_to_kg(M_co2)
#     delta_A_co2 = 0.1
#     if time_horizon==20:
#         I_co2 = 14.2
#         delta_I_co2 = (16.3-12.2)/2/I_co2
#     elif time_horizon==100:
#         I_co2 = 52.4 
#         delta_I_co2 = (65.2-39.5)/2/I_co2
#     elif time_horizon==500:
#         I_co2 = 184
#         delta_I_co2 = (235-132)/2/I_co2
#     co2_agwp     = A_co2*I_co2
#     co2_ddelta_A = I_co2*delta_A_co2*A_co2
#     co2_ddelta_I = A_co2*delta_I_co2*I_co2
#     co2_delta_agwp = compute_delta_f(co2_agwp, co2_ddelta_A, co2_ddelta_I)
#     conf_level = 0.9
#     co2_std_agwp = conf_interval_to_std(conf_level, co2_delta_agwp)
#     return co2_delta_agwp, co2_std_agwp, co2_agwp
# co2_delta_agwp, co2_std_agwp, co2_agwp = compute_co2_agwp(H)
# print_uncertainties('CO2 agwp', co2_delta_agwp, co2_std_agwp)

# # 2. Uncertainty values for AGWP, chapter 8.SM.12, equation 8.SM.24
# # delta values for A and tau are taken from table 8.SM.12
# compute_gwp          = lambda x_agwp: x_agwp/co2_agwp
# compute_dgwp_dA      = lambda A, tau, mult, delta_A: \
#     mult * tau / co2_agwp    * (1 - np.exp(-H/tau)) * delta_A*A
# compute_dgwp_dtau    = lambda A, tau, mult, delta_tau: \
#     mult *   A / co2_agwp    * (1 - np.exp(-H/tau) - H/tau*np.exp(-H/tau) ) * delta_tau*tau
# compute_dgwp_dgwpco2 = lambda x_agwp: \
#     1 / co2_agwp**2 * x_agwp * co2_delta_agwp*co2_agwp

# def compute_delta_gwp(x_name, M, A, tau, delta_A, delta_tau, mult=1, *dgwp_dy):
#     x_agwp         = compute_agwp_general (A, tau, mult)
#     x_gwp          = compute_gwp(x_agwp)
#     x_dgwp_dA      = compute_dgwp_dA  (A, tau, mult, delta_A)
#     x_dgwp_dtau    = compute_dgwp_dtau(A, tau, mult, delta_tau)
#     x_dgwp_dgwpco2 = compute_dgwp_dgwpco2(x_agwp)
#     x_delta_gwp = compute_delta_f(
#         x_gwp,
#         x_dgwp_dA, 
#         x_dgwp_dtau, 
#         x_dgwp_dgwpco2,
#         *dgwp_dy,
#     )
#     conf_level = 0.9
#     x_std_multiplier_gwp = conf_interval_to_std(conf_level, x_delta_gwp)
#     print_uncertainties(x_name, x_delta_gwp, x_std_multiplier_gwp)
#     return x_delta_gwp, x_std_multiplier_gwp

# Methane CH4, AGWP taken from chapter 8.SM.11.3.2, equation 8.SM.19
M_ch4 = 16.04
A_ch4 = 3.63e-4 * ppbv_to_kg(M_ch4)
tau  = 12.4
f1   = 0.5   # due to effects on ozone
f2   = 0.15  # due to stratospheric H2O
mult = 1+f1+f2
delta_A   = 0.1
delta_tau = 0.1857
delta_f1  = 0.6
delta_f2  = 0.7143
ch4_dgwp_df1  = A_ch4*tau / co2_agwp * (1 - np.exp(-H/tau)) * delta_f1*f1
ch4_dgwp_df2  = A_ch4*tau / co2_agwp * (1 - np.exp(-H/tau)) * delta_f2*f2
ch4_delta_gwp, ch4_std_multiplier_gwp = compute_delta_gwp(
    'CH4', M_ch4, A_ch4, tau, delta_A, delta_tau, mult, ch4_dgwp_df1, ch4_dgwp_df2
)
ghg_std_multiplier_dict['Methane, fossil'] = ch4_std_multiplier_gwp
ghg_std_multiplier_dict['Methane, non-fossil'] = ch4_std_multiplier_gwp
ghg_std_multiplier_dict['Methane, from soil or biomass stock'] = ch4_std_multiplier_gwp

# Dinitrogen monoxide, table 8.A.1
M = 44.013
A = 3.00e-3 * ppbv_to_kg(M)
tau = 121
f1 = 0.5   # due to effects on ozone
f2 = 0.15  # due to stratospheric H2O
RE_ch4 = A_ch4
RE_n2o = A
mult = 1-0.36*(1+f1+f2)*RE_ch4/RE_n2o
delta_A = 0.1
delta_tau = 0.1299
delta_f1 = 0.6
delta_f2 = 0.7143
n2o_dgwp_df1     = -0.36*RE_ch4/RE_n2o*A*tau / co2_agwp * (1 - np.exp(-H/tau)) * delta_f1*f1
n2o_dgwp_df2     = -0.36*RE_ch4/RE_n2o*A*tau / co2_agwp * (1 - np.exp(-H/tau)) * delta_f2*f2
n2o_delta_gwp, n2o_std_multiplier_gwp = compute_delta_gwp(
    'N2O', M, A, tau, delta_A, delta_tau, mult, n2o_dgwp_df1, n2o_dgwp_df2
)
ghg_std_multiplier_dict['Dinitrogen monoxide'] = n2o_std_multiplier_gwp

# Carbon monoxide, table 8.A.4, doi:10.1016/j.atmosenv.2009.04.044
# ghg_std_multiplier_dict['Carbon monoxide'] = nox_std_multiplier_gwp


# # CFC-11
# M = 137.37
# A = 0.26 * ppbv_to_kg(M)
# tau = 45
# mult = 1
# delta_A = 0.1
# delta_tau = 0.2255
# cfc11_agwp         = compute_agwp_general (A, tau, mult)
# cfc11_gwp          = compute_gwp(cfc11_agwp)
# cfc11_dgwp_dA      = compute_dgwp_dA  (A, tau, mult, delta_A)
# cfc11_dgwp_dtau    = compute_dgwp_dtau(A, tau, mult, delta_tau)
# cfc11_dgwp_dgwpco2 = compute_dgwp_dgwpco2(cfc11_agwp)
# cfc11_delta_gwp, cfc11_std_multiplier_gwp = compute_delta_gwp('CFC-11', M, A, tau, delta_A, delta_tau, mult)

# # CFC-12
# M = 120.91
# A = 0.32 * ppbv_to_kg(M)
# tau = 100
# mult = 1
# delta_A = 0.1
# delta_tau = 0.2876
# cfc12_delta_gwp, cfc12_std_multiplier_gwp = compute_delta_gwp('CFC-12', M, A, tau, delta_A, delta_tau, mult)

# # Ethane, 1,1,1,2-tetrafluoro-, HFC-134a
# M = 102.03
# A = 0.16 * ppbv_to_kg(M)
# tau = 13.4
# mult = 1
# delta_A = 0.1
# delta_tau = 0.179
# hfc134a_delta_gwp, hfc134a_std_multiplier_gwp = compute_delta_gwp('HFC-134a', M, A, tau, delta_A, delta_tau, mult)
# ghg_std_multiplier_dict['Ethane, 1,1,1,2-tetrafluoro-, HFC-134a'] = hfc134a_std_multiplier_gwp

# Sulfur hexafluoride
M = 146.06
A = 0.57 * ppbv_to_kg(M)
tau = 3200
mult = 1
delta_A = 0.1   # TODO chosen arbitrarily
delta_tau = 0.2 # TODO chosen arbitrarily
sf6_delta_gwp, sf6_std_multiplier_gwp = compute_delta_gwp('SF6', M, A, tau, delta_A, delta_tau, mult)
ghg_std_multiplier_dict['Sulfur hexafluoride'] = sf6_std_multiplier_gwp

# # Ethane, hexafluoro-, HFC-116, chemical formula C2F6
# M = 138.01
# A = 0.25 * ppbv_to_kg(M)
# tau = 10000
# mult = 1
# delta_A = 0.1   # TODO chosen arbitrarily
# delta_tau = 0.2 # TODO chosen arbitrarily
# hfc116_delta_gwp, hfc116_std_multiplier_gwp = compute_delta_gwp('HFC-116', M, A, tau, delta_A, delta_tau, mult)
# ghg_std_multiplier_dict['Ethane, hexafluoro-, HFC-116'] = hfc116_std_multiplier_gwp

# # Methane, tetrafluoro-, R-14
# M = 88.0043
# A = 0.09 * ppbv_to_kg(M)
# tau = 50000
# mult = 1
# delta_A = 0.1   # TODO chosen arbitrarily
# delta_tau = 0.2 # TODO chosen arbitrarily
# r14_delta_gwp, r14_std_multiplier_gwp = compute_delta_gwp('R-14', M, A, tau, delta_A, delta_tau, mult)
# ghg_std_multiplier_dict['Methane, tetrafluoro-, R-14'] = r14_std_multiplier_gwp

# # Methane, bromotrifluoro-, Halon 1301
# M = 148.91
# A = 0.3 * ppbv_to_kg(M)
# tau = 65
# mult = 1
# delta_A = 0.1   # TODO chosen arbitrarily
# delta_tau = 0.2 # TODO chosen arbitrarily
# halon1301_delta_gwp, halon1301_std_multiplier_gwp = compute_delta_gwp('Halon-1301', M, A, tau, delta_A, delta_tau, mult)
# ghg_std_multiplier_dict['Methane, bromotrifluoro-, Halon 1301'] = halon1301_std_multiplier_gwp

# New bw method with uncertainties

In [ ]:
import stats_arrays as sa
bw_method = bd.Method(method)

flows_certain_list, flows_uncertain_list = [], []
for flow in bw_method.load():
    if flow[0] not in codes:
        flows_certain_list.append(flow)
    else:
        act = bd.get_activity(flow[0])
        if 'Carbon dioxide' in act['name']:
            flows_certain_list.append(flow)
        elif 'Carbon monoxide, fossil' in act['name'] or \
             'Carbon monoxide, from soil or biomass stock' in act['name']:
            oxidation = 1.5714
            min_ = 2 + oxidation
            max_ = 3.3 + oxidation
            flow_uncertain = (
                flow[0], # flow key = (database, code)
                {
                    'amount': flow[1], 
                    'minimum': min_, 
                    'maximum': max_, 
                    'uncertainty type': sa.UniformUncertainty.id, # assumption from the ipcc report
                }
            )
            flows_uncertain_list.append(flow_uncertain)
        else:
            ghg_gwp = flow[1]
            ghg_std = ghg_gwp*ghg_std_multiplier_dict[act['name']]
            flow_uncertain = (
                flow[0], # flow key = (database, code)
                {
                    'amount': ghg_gwp, # static value of gwp
                    'uncertainty type': sa.NormalUncertainty.id, # assumption from the ipcc report
                    'loc': ghg_gwp, # mean value that is equal to the static one
                    'scale': ghg_std, # standard deviation
                }
            )
            flows_uncertain_list.append(flow_uncertain)
flows_list = flows_certain_list + flows_uncertain_list

In [ ]:
uncertain_method = method + ('uncertain',)
uncertain_bw_method = bd.Method(uncertain_method)
try:
    uncertain_bw_method.deregister()
except:
    pass
uncertain_bw_method.register(
    **{
        'unit': bw_method.metadata['unit'],
        'num_cfs':len(flows_list),
        'abbreviation':'nonexistent',
        'description':'based on IPCC 2013 method but with uncertainties',
        'filename':'nonexistent'
    }
)
uncertain_bw_method.write(flows_list)

In [ ]:
lca = bc.LCA(demand, uncertain_method)
lca.lci()
lca.lcia()
lca.score

In [ ]:
# uncertain_tech = lca.tech_params[lca.tech_params['uncertainty_type']>1]
# uncertain_bio = lca.bio_params[lca.bio_params['uncertainty_type']>1]
# uncertain_cf = lca.cf_params[lca.cf_params['uncertainty_type']>1]
# num_params = len(uncertain_tech) + len(uncertain_bio) + len(uncertain_cf)
# len(uncertain_tech), len(uncertain_bio), len(uncertain_cf)

In [ ]:
# iterations = 20
# path_base = Path("/Users/akim/PycharmProjects/gsa_framework/dev/write_files/")
# # path_base = Path('/data/user/kim_a/paper_gsa')
# write_dir = path_base / "protocol_gsa" / "lca_model_food_num_params".format(num_params)
# if flag_generate_scores_dict:
#     model = LCAModel(demand, method, write_dir)  # generate scores_dict
#     del model
# model = LCAModel(demand, method, write_dir, num_params=num_params)
# gsa_seed = 92374523

In [ ]:
%%time
iterations = 4000
mc_certain_gwp = bc.ParallelMonteCarlo(demand, method, iterations)
lca_scores_certain_gwp = np.array(mc_certain_gwp.calculate())

mc_uncertain_gwp = bc.ParallelMonteCarlo(demand, uncertain_method, iterations)
lca_scores_uncertain_gwp = np.array(mc_uncertain_gwp.calculate())

In [ ]:
# %%time
# iterations = 200
# seed = 234534
# mc_certain_gwp1 = bc.MonteCarloLCA(demand, method, seed=seed)
# lca_scores_certain_gwp = [next(mc_certain_gwp1) for _ in range(iterations)]

# mc_uncertain_gwp1 = bc.MonteCarloLCA(demand, uncertain_method, seed=seed)
# lca_scores_uncertain_gwp = [next(mc_uncertain_gwp1) for _ in range(iterations)]

In [ ]:
np.std(lca_scores_certain_gwp), np.std(lca_scores_uncertain_gwp)

# Plot LCA scores

In [ ]:
num_bins = 60
bin_min = min(np.hstack([lca_scores_certain_gwp, lca_scores_uncertain_gwp]))
bin_max = max(np.hstack([lca_scores_certain_gwp, lca_scores_uncertain_gwp]))
bins_ = np.linspace(bin_min, bin_max, num_bins, endpoint=True)
freq_certain, bins_certain = np.histogram(lca_scores_certain_gwp, bins=bins_, density=True)
freq_uncertain, bins_uncertain = np.histogram(lca_scores_uncertain_gwp, bins=bins_, density=True)
    
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=bins_certain,
        y=freq_certain,
        name='GWP values W/O uncertainties',
        opacity=0.7,
    ),
)
fig.add_trace(
    go.Bar(
        x=bins_uncertain,
        y=freq_uncertain,
        name='GWP values WITH uncertainties',
        opacity=0.7,
    ),
)
fig.update_layout(barmode="overlay")
fig.update_yaxes(title_text='Frequency')
fig.update_xaxes(title_text='LCA scores, [kg CO2-eq]')
fig.show()
fig.write_html("write_files/lca_scores_gwp_uncertainties.html")

# Plot GWP with uncertainties

In [ ]:
data = []
flow_names = []
iterations = 2000
for flow in flows_uncertain_list:
    if flow[1]['uncertainty type'] == sa.NormalUncertainty.id:
        x = np.random.normal(flow[1]['loc'], flow[1]['scale'], iterations)
    elif flow[1]['uncertainty type'] == sa.UniformUncertainty.id:
        x = (flow[1]['maximum'] - flow[1]['minimum'])*np.random.rand(iterations) + flow[1]['minimum']
    act = bd.get_activity(flow[0])
    flow_name = "{} {}".format(act['name'], act['categories'])[:45]
    data.append(
        {
            'name': flow_name,
            'static': flow[1]['amount'],
            'x': x
        }
    )
    flow_names.append(flow_name)

In [ ]:
n_plots = len(flows_uncertain_list)
ncols = 4
nrows = int(np.ceil(n_plots/ncols))
fig = make_subplots(
    rows=nrows,
    cols=ncols,
    shared_yaxes=True,
    subplot_titles=flow_names,
)
i = 0
for row in range(nrows):
    for col in range(ncols):
        if i < n_plots:
            values = data[i]['x']
            freq, bins = np.histogram(values, bins=num_bins)
            fig.add_trace(
                go.Bar(
                    x=bins,
                    y=freq,
                    showlegend=False,
                    marker_color = 'blue',
                    opacity=0.7,
                ),
                row=row+1,
                col=col+1,
            )
            fig.add_trace(
                go.Scatter(
                    x=[data[i]['static']],
                    y=[0],
                    showlegend=False,
                    marker_color = 'red',
                    mode="markers",
                    marker_symbol = 'x',
                    marker_size=15,
                    opacity=0.7,
                ),
                row=row+1,
                col=col+1,
            )
            i += 1 
fig.update_yaxes(title_text="Frequency", col=1)
fig.update_xaxes(title_text="GWP", row=nrows)
fig.update_layout(
    width=400*ncols, height=300*nrows,
)
fig.show()
fig.write_html("write_files/gwp_uncertainties.html")